In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# dx = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
# dx.head()

In [ ]:
# let's see data first
# df.describe()

In [ ]:
from sklearn import model_selection
from sklearn import tree
from sklearn import metrics

In [ ]:
# Create fold using k-cross
def create_fold(data):
    data['kfold'] = -1
    data = data.sample(frac = 1).reset_index(drop = True)
    
    kf = model_selection.StratifiedKFold(n_splits = 6)
    
    for fold , (t_ , v_) in enumerate(kf.split(X = data , y = data.label.values)):
        data.loc[v_ , 'kfold'] = fold
    
    return data


In [ ]:
def run(data , fold , depth):
    df_train = data[data.kfold != fold].reset_index(drop = True)
    df_val = data[data.kfold == fold].reset_index(drop = True)
    
    X_train = df_train.drop('label',axis=1).values
    y_train = df_train.label.values
    
    X_val = df_val.drop('label' , axis = 1).values
    y_val = df_val.label.values
    
    model = tree.DecisionTreeClassifier(max_depth = depth)
    
    model.fit(X_train , y_train)
    
    y_pred = model.predict(X_val)
    
    print(f"fold : {fold} , depth : {depth} , Accuracy : {metrics.accuracy_score(y_val , y_pred)}")
    

In [ ]:
if __name__ == '__main__':
    # read data
    df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    
    #create fold
#     df = create_fold(df)
    
    # run for all folds
#     for depth in [8,9,10,11,12,13,14,15]:
#         for i in range(6):
#             run(df , i , depth)

    # so depth 15 is best
    model = tree.DecisionTreeClassifier(max_depth = 15)
    
    X = df.drop('label',axis=1).values
    y = df.label.values
    model.fit(X,y)
    
    df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
    
    df_pred = model.predict(df_test)
    output = pd.DataFrame({'ImageId' : df_test.index + 1, 'label' : df_pred})
#     print(output.head())
    output.to_csv('my_submission.csv' , index=False)